# Weather Analysis for Lexington, KY
### Data Cleaning Notebook

The goal of this notebook is to go through all of the csv data files that I downloaded from NOAA's website.  We'll explore the data some, remove or fill in NaN values, create labels for certain tables and then combine them as needed.  Once we're done with that, we'll save the data to a database that we'll be able to use later on.  We will also save the final dataframe(s) to new csv files to hopefully create a tableau dashboard.

In [7]:
import pandas as pd
import os
import sqlite3

In [8]:
current_data = os.path.join('data', 'current_2000_weather.csv')
historical_data = os.path.join('data', 'historical_weather.csv')
dailies_2025 = os.path.join('data', 'dailies_2010_2025.csv')
dailies_2000 = os.path.join('data', 'dailies_2000_2009.csv')

df_current = pd.read_csv(current_data)
df_historical = pd.read_csv(historical_data)
df_dailies_2025 = pd.read_csv(dailies_2025)
df_dailies_2000 = pd.read_csv(dailies_2000)

In [46]:
# I want to see all of the columns in the current data
pd.set_option('display.max_columns', None)

df_current.head()

,STATION,NAME,LATITUDE,LONGITUDE,ELEVATION,DATE,DEWP,DEWP_ATTRIBUTES,FRSHTT,FRSHTT_ATTRIBUTES,GUST,GUST_ATTRIBUTES,MAX,MAX_ATTRIBUTES,MIN,MIN_ATTRIBUTES,MXSPD,MXSPD_ATTRIBUTES,PRCP,PRCP_ATTRIBUTES,SLP,SLP_ATTRIBUTES,SNDP,SNDP_ATTRIBUTES,STP,STP_ATTRIBUTES,TEMP,TEMP_ATTRIBUTES,VISIB,VISIB_ATTRIBUTES,WDSP,WDSP_ATTRIBUTES
0,72422093820,"LEXINGTON BLUEGRASS AIRPORT, KY US",38.03391,-84.61138,293.2,2000-01-01,30.0,24,0,NaN,999.9,NaN,57.9,,30.0,,13.0,NaN,0.00,G,1020.3,24,999.9,NaN,999.9,0,46.1,24,9.7,24,6.8,24
1,72422093820,"LEXINGTON BLUEGRASS AIRPORT, KY US",38.03391,-84.61138,293.2,2000-01-02,44.9,24,0,NaN,33.0,NaN,64.0,,37.9,,25.1,NaN,0.00,G,1016.5,23,999.9,NaN,999.9,0,56.7,24,8.9,24,14.1,24
2,72422093820,"LEXINGTON BLUEGRASS AIRPORT, KY US",38.03391,-84.61138,293.2,2000-01-03,54.1,24,110000,NaN,27.0,NaN,68.0,,52.0,,22.0,NaN,1.31,G,1012.5,21,999.9,NaN,999.9,0,58.6,24,8.2,24,11.5,24
3,72422093820,"LEXINGTON BLUEGRASS AIRPORT, KY US",38.03391,-84.61138,293.2,2000-01-04,42.9,24,111000,NaN,34.0,NaN,68.0,,30.0,,22.0,NaN,1.07,G,1010.0,20,999.9,NaN,999.9,0,48.2,24,9.2,24,12.4,24
4,72422093820,"LEXINGTON BLUEGRASS AIRPORT, KY US",38.03391,-84.61138,293.2,2000-01-05,20.5,24,1000,NaN,19.0,NaN,32.0,*,26.6,*,13.0,NaN,0.08,G,1027.4,19,999.9,NaN,999.9,0,28.4,24,9.7,24,7.0,24


The main data I downloaded does not include any MIN or MAX of temperature.  Only the Average for the day.  I went to NOAA's site to fetch daily information.  This is a consolidated csv file with data from 54 weather stations in a certain radius around Lexington.  I had to download this into two files because of limitations.  

In [47]:
df_dailies_2000.head()

,STATION,NAME,DATE,AWND,DAPR,FMTM,MDPR,PGTM,PRCP,SNOW,SNWD,TAVG,TMAX,TMIN,WDF2,WDF5,WSF2,WSF5
7281,USW00093820,"LEXINGTON BLUEGRASS AIRPORT, KY US",2000-01-01,9.17,NaN,430.0,NaN,405.0,0.00,0.0,0.0,48.0,58.0,38.0,130.0,130.0,15.0,17.9
7282,USW00093820,"LEXINGTON BLUEGRASS AIRPORT, KY US",2000-01-02,17.22,NaN,1142.0,NaN,1134.0,0.25,0.0,0.0,58.0,64.0,52.0,210.0,220.0,33.1,40.0
7283,USW00093820,"LEXINGTON BLUEGRASS AIRPORT, KY US",2000-01-03,13.65,NaN,2022.0,NaN,2018.0,2.05,0.0,0.0,61.0,68.0,53.0,190.0,180.0,38.9,48.1
7284,USW00093820,"LEXINGTON BLUEGRASS AIRPORT, KY US",2000-01-04,12.97,NaN,2020.0,NaN,2019.0,0.08,0.0,0.0,42.0,57.0,27.0,290.0,290.0,19.9,25.1
7285,USW00093820,"LEXINGTON BLUEGRASS AIRPORT, KY US",2000-01-05,6.04,NaN,448.0,NaN,447.0,0.00,0.0,0.0,29.0,32.0,25.0,290.0,290.0,14.1,16.1


In [48]:
df_dailies_2025.head()

,STATION,NAME,DATE,AWND,DAPR,FMTM,MDPR,PGTM,PRCP,SNOW,SNWD,TAVG,TMAX,TMIN,WDF2,WDF5,WSF2,WSF5
15629,USW00063838,"VERSAILLES 3 NNW, KY US",2010-01-01,NaN,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN,35.0,16.0,NaN,NaN,NaN,NaN
15630,USW00063838,"VERSAILLES 3 NNW, KY US",2010-01-02,NaN,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN,21.0,11.0,NaN,NaN,NaN,NaN
15631,USW00063838,"VERSAILLES 3 NNW, KY US",2010-01-03,NaN,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN,24.0,5.0,NaN,NaN,NaN,NaN
15632,USW00063838,"VERSAILLES 3 NNW, KY US",2010-01-04,NaN,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN,27.0,17.0,NaN,NaN,NaN,NaN
15633,USW00063838,"VERSAILLES 3 NNW, KY US",2010-01-05,NaN,NaN,NaN,NaN,NaN,0.02,NaN,NaN,NaN,22.0,12.0,NaN,NaN,NaN,NaN


Looking at the data here, and based on the information from NOAA's site, there are a lot of NaN / missing data.  I am going to loop through both of these dataframes and remove any row that has NaN in **both** the TMAX and TMIN columns.  I initially hoped to use this data for more info on precipitation and wind data, but I think for now I will focus on just isolating the temp data.

In [50]:
# Removing any row that does not have data in the TMIN and TMAX columns
def rm_data_without_temp(df):
    """
    Removes any row that does not have data in both the TMIN and TMAX columns
    """
    return df[df['TMIN'].notna() & df['TMAX'].notna()]

df_dailies_2000 = rm_data_without_temp(df_dailies_2000)
df_dailies_2000.head()

,STATION,NAME,DATE,AWND,DAPR,FMTM,MDPR,PGTM,PRCP,SNOW,SNWD,TAVG,TMAX,TMIN,WDF2,WDF5,WSF2,WSF5
7281,USW00093820,"LEXINGTON BLUEGRASS AIRPORT, KY US",2000-01-01,9.17,NaN,430.0,NaN,405.0,0.00,0.0,0.0,48.0,58.0,38.0,130.0,130.0,15.0,17.9
7282,USW00093820,"LEXINGTON BLUEGRASS AIRPORT, KY US",2000-01-02,17.22,NaN,1142.0,NaN,1134.0,0.25,0.0,0.0,58.0,64.0,52.0,210.0,220.0,33.1,40.0
7283,USW00093820,"LEXINGTON BLUEGRASS AIRPORT, KY US",2000-01-03,13.65,NaN,2022.0,NaN,2018.0,2.05,0.0,0.0,61.0,68.0,53.0,190.0,180.0,38.9,48.1
7284,USW00093820,"LEXINGTON BLUEGRASS AIRPORT, KY US",2000-01-04,12.97,NaN,2020.0,NaN,2019.0,0.08,0.0,0.0,42.0,57.0,27.0,290.0,290.0,19.9,25.1
7285,USW00093820,"LEXINGTON BLUEGRASS AIRPORT, KY US",2000-01-05,6.04,NaN,448.0,NaN,447.0,0.00,0.0,0.0,29.0,32.0,25.0,290.0,290.0,14.1,16.1


In [14]:
df_dailies_2025 = rm_data_without_temp(df_dailies_2025)
df_dailies_2025.head()

,STATION,NAME,DATE,AWND,DAPR,FMTM,MDPR,PGTM,PRCP,SNOW,SNWD,TAVG,TMAX,TMIN,WDF2,WDF5,WSF2,WSF5
15629,USW00063838,"VERSAILLES 3 NNW, KY US",2010-01-01,NaN,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN,35.0,16.0,NaN,NaN,NaN,NaN
15630,USW00063838,"VERSAILLES 3 NNW, KY US",2010-01-02,NaN,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN,21.0,11.0,NaN,NaN,NaN,NaN
15631,USW00063838,"VERSAILLES 3 NNW, KY US",2010-01-03,NaN,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN,24.0,5.0,NaN,NaN,NaN,NaN
15632,USW00063838,"VERSAILLES 3 NNW, KY US",2010-01-04,NaN,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN,27.0,17.0,NaN,NaN,NaN,NaN
15633,USW00063838,"VERSAILLES 3 NNW, KY US",2010-01-05,NaN,NaN,NaN,NaN,NaN,0.02,NaN,NaN,NaN,22.0,12.0,NaN,NaN,NaN,NaN


I noticed that some data is from the Bluegrass Airport, which is where all of the main data is from.  I just want to see how many of the rows are from the airport in both of these dailies dataframes...

In [15]:
LEX_BG_SATATION = 'USW00093820'
filtered_df = df_dailies_2025[df_dailies_2025['STATION'] == LEX_BG_SATATION]
filtered_df.head()

,STATION,NAME,DATE,AWND,DAPR,FMTM,MDPR,PGTM,PRCP,SNOW,SNWD,TAVG,TMAX,TMIN,WDF2,WDF5,WSF2,WSF5
61204,USW00093820,"LEXINGTON BLUEGRASS AIRPORT, KY US",2010-01-01,9.84,NaN,558.0,NaN,611.0,0.01,0.0,0.0,NaN,34.0,17.0,290.0,290.0,16.1,21.9
61205,USW00093820,"LEXINGTON BLUEGRASS AIRPORT, KY US",2010-01-02,9.17,NaN,401.0,NaN,1417.0,0.01,0.0,0.0,NaN,20.0,12.0,310.0,270.0,17.0,23.0
61206,USW00093820,"LEXINGTON BLUEGRASS AIRPORT, KY US",2010-01-03,7.61,NaN,1206.0,NaN,1402.0,0.00,0.0,0.0,NaN,23.0,6.0,320.0,330.0,16.1,19.9
61207,USW00093820,"LEXINGTON BLUEGRASS AIRPORT, KY US",2010-01-04,10.07,NaN,1648.0,NaN,1653.0,0.00,0.1,0.0,NaN,26.0,17.0,280.0,290.0,16.1,19.9
61208,USW00093820,"LEXINGTON BLUEGRASS AIRPORT, KY US",2010-01-05,10.07,NaN,2136.0,NaN,2136.0,0.01,0.6,0.0,NaN,21.0,12.0,280.0,280.0,16.1,19.9


In [16]:
len(filtered_df)

5689

In [17]:
filtered_df_2000 = df_dailies_2000[df_dailies_2000['STATION'] == 'USW00093820']

filtered_df_2000.head()

,STATION,NAME,DATE,AWND,DAPR,FMTM,MDPR,PGTM,PRCP,SNOW,SNWD,TAVG,TMAX,TMIN,WDF2,WDF5,WSF2,WSF5
7281,USW00093820,"LEXINGTON BLUEGRASS AIRPORT, KY US",2000-01-01,9.17,NaN,430.0,NaN,405.0,0.00,0.0,0.0,48.0,58.0,38.0,130.0,130.0,15.0,17.9
7282,USW00093820,"LEXINGTON BLUEGRASS AIRPORT, KY US",2000-01-02,17.22,NaN,1142.0,NaN,1134.0,0.25,0.0,0.0,58.0,64.0,52.0,210.0,220.0,33.1,40.0
7283,USW00093820,"LEXINGTON BLUEGRASS AIRPORT, KY US",2000-01-03,13.65,NaN,2022.0,NaN,2018.0,2.05,0.0,0.0,61.0,68.0,53.0,190.0,180.0,38.9,48.1
7284,USW00093820,"LEXINGTON BLUEGRASS AIRPORT, KY US",2000-01-04,12.97,NaN,2020.0,NaN,2019.0,0.08,0.0,0.0,42.0,57.0,27.0,290.0,290.0,19.9,25.1
7285,USW00093820,"LEXINGTON BLUEGRASS AIRPORT, KY US",2000-01-05,6.04,NaN,448.0,NaN,447.0,0.00,0.0,0.0,29.0,32.0,25.0,290.0,290.0,14.1,16.1


In [18]:
len(filtered_df_2000)

3653

Based on the length for each of them, it looks like it *probably* covers every day, bar a few missing.  I am going to combine these two dataframes with each other, and potentially narrow the data down to using just the data from the Bluegrass Airport, unless it doesn't exist, in which use the next available data.

In [52]:
# Checking that dates exist & combining dataframes (optional)

def complete_date_range(
        df_primary, 
        df_secondary=None,
        airport_station: str = 'USW00093820',
        start_date: str = "2000-01-01",
        end_date: str = "2025-08-03"):
    """
    Checks if the date range of the primary dataframe is complete.
    If more than one dataframe is provided, it will be combined with the first dataframe and then checked for missing dates.

    Args:
        df_primary (pd.DataFrame): The primary dataframe to check.
        df_secondary (pd.DataFrame, optional): The secondary dataframe to use for filling in missing dates.
        airport_station (str, optional): The station number of the airport. (can change this to any station, not just an airport)
        start_date (str, optional): The start date of the date range. (YYYY-MM-DD)
        end_date (str, optional): The end date of the date range. (YYYY-MM-DD)

    Returns:
        pd.DataFrame: A dataframe with the complete date range.
    """
    # Convert date columns to datetime
    df_primary['DATE'] = pd.to_datetime(df_primary['DATE'], format='%Y-%m-%d')
    df_primary = df_primary.sort_values(by='DATE')

    if df_secondary is not None:
        df_secondary['DATE'] = pd.to_datetime(df_secondary['DATE'], format='%Y-%m-%d')
        df_secondary = df_secondary.sort_values(by='DATE')

        # combining the two DFs
        combined_df = pd.concat([df_primary, df_secondary], ignore_index=True)
        combined_df = combined_df.sort_values(by='DATE')
    else:
        combined_df = df_primary

    ap_dates = set(combined_df[combined_df['STATION'] == airport_station]['DATE'].dt.date)
    bu_dates = set(combined_df[combined_df['STATION'] != airport_station]['DATE'].dt.date)
    expected_dates = set(pd.date_range(start=start_date, end=end_date, freq='D').date)
    missing_dates = (bu_dates - ap_dates) - expected_dates
    
    print(f"Missing dates: {missing_dates}")

    ## FUTURE TODO: Separate the DF into airport DF and everything else DF, then fill in the missing dates with the other DF.  For now we'll use it all.
    # Next we'll separate the main dataframe (again) into airport DF and everything else DF
    # df_airport = combined_df[combined_df['STATION'] == airport_station]
    # df_other = combined_df[combined_df['STATION'] != airport_station]

    if len(missing_dates) > 0:
        print("Dates are missing from the data.  Please keep in mind the folowing dates are missing:")
        print(missing_dates)
    else:
        print("All dates are present in the data.")

    # Making sure the data is sorted by date
    combined_df = combined_df.sort_values(by='DATE')

    print("Successfully completed combining the dataframes.")
    return combined_df


df_dailies = complete_date_range(df_dailies_2000, df_dailies_2025)



Missing dates: set()
All dates are present in the data.
Successfully completed combining the dataframes.


In [ ]:
# checking & sorting the historical data

df_historical.sort_values(by='DATE')

,STATION,NAME,LATITUDE,LONGITUDE,ELEVATION,DATE,DEWP,DEWP_ATTRIBUTES,FRSHTT,FRSHTT_ATTRIBUTES,GUST,GUST_ATTRIBUTES,MAX,MAX_ATTRIBUTES,MIN,MIN_ATTRIBUTES,MXSPD,MXSPD_ATTRIBUTES,PRCP,PRCP_ATTRIBUTES,SLP,SLP_ATTRIBUTES,SNDP,SNDP_ATTRIBUTES,STP,STP_ATTRIBUTES,TEMP,TEMP_ATTRIBUTES,VISIB,VISIB_ATTRIBUTES,WDSP,WDSP_ATTRIBUTES
9861,99999993820,"LEXINGTON BLUEGRASS AIRPORT, KY US",38.04080,-84.60580,298.7,1948-01-01,47.3,13,10000,NaN,999.9,NaN,64.0,*,37.9,*,26.0,NaN,99.99,,994.7,13,999.9,NaN,959.0,13,53.5,13,11.6,13,19.9,13
9862,99999993820,"LEXINGTON BLUEGRASS AIRPORT, KY US",38.04080,-84.60580,298.7,1948-01-02,27.8,24,111000,NaN,999.9,NaN,35.1,*,26.1,*,27.0,NaN,99.99,,1004.9,24,999.9,NaN,968.1,24,29.1,24,5.3,24,19.2,24
9863,99999993820,"LEXINGTON BLUEGRASS AIRPORT, KY US",38.04080,-84.60580,298.7,1948-01-03,28.6,24,111000,NaN,999.9,NaN,30.9,*,28.0,*,11.1,NaN,99.99,,1018.4,24,999.9,NaN,981.0,24,29.4,24,3.0,24,6.2,24
9864,99999993820,"LEXINGTON BLUEGRASS AIRPORT, KY US",38.04080,-84.60580,298.7,1948-01-04,29.6,24,111000,NaN,999.9,NaN,35.1,*,30.0,*,22.0,NaN,99.99,,1013.2,24,999.9,NaN,976.0,24,32.4,24,7.1,24,12.8,24
9865,99999993820,"LEXINGTON BLUEGRASS AIRPORT, KY US",38.04080,-84.60580,298.7,1948-01-05,27.8,24,1000,NaN,999.9,NaN,34.0,*,28.0,*,15.9,NaN,99.99,,1023.2,24,999.9,NaN,985.6,24,30.9,24,6.7,24,7.9,24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9856,72422093820,"LEXINGTON BLUEGRASS AIRPORT, KY US",38.03391,-84.61138,293.2,1999-12-27,17.2,24,101000,NaN,999.9,NaN,39.0,,19.0,,11.1,NaN,0.00,G,1017.4,23,999.9,NaN,999.9,0,24.8,24,8.9,24,7.0,24
9857,72422093820,"LEXINGTON BLUEGRASS AIRPORT, KY US",38.03391,-84.61138,293.2,1999-12-28,18.3,24,101000,NaN,27.0,NaN,30.9,,17.1,,21.0,NaN,0.01,G,1012.1,19,999.9,NaN,999.9,0,23.2,24,8.7,24,7.5,24
9858,72422093820,"LEXINGTON BLUEGRASS AIRPORT, KY US",38.03391,-84.61138,293.2,1999-12-29,24.6,24,0,NaN,999.9,NaN,39.0,,17.1,,15.0,NaN,0.00,G,1013.9,24,999.9,NaN,999.9,0,32.0,24,9.9,24,6.8,24
9859,72422093820,"LEXINGTON BLUEGRASS AIRPORT, KY US",38.03391,-84.61138,293.2,1999-12-30,31.6,24,0,NaN,999.9,NaN,52.0,,28.9,,14.0,NaN,0.00,G,1013.2,24,999.9,NaN,999.9,0,41.7,24,9.9,24,9.3,24


So now I have three data frames:

    - Dailes DF: has daily information from 2000 - 2025
    - df_current: data from 2000 - 2025, without TMAX TMIN
    - df_historical: data from 1948 - 2000

Most data is from the Bluegrass Airport; some of the daily data is from areas around Lexington.

So the data I would like to focus on for now are the Temperature trends (MIN MAX AVG)

I'd like to also consider percipitation and wind data down the line.

While also including the data and location data!

In [22]:
# Two lists of columns I want to keep and their information
# More for reference than anything else.

core_columns =[
    'STATION',
    'NAME',
    'DATE',
    'TEMP', # Temp data from current/historical data
    'TAVG', # Average temp data from dailies - not always available but we could calculate it
    'TMAX', # Dailies data
    'TMIN', # Dailies data
]
secondary_columns = [
    'PRCP', # Percipitation - available in dailies data as well as current/historical data
    'MAX', # Max GUST speed - available in current/historical data - I'd like to RENAME these to GUST_MAX and GUST_MIN
    'MIN', # Min GUST speed - available in current/historical data
    'WDSP', # Wind speed - available in current/historical data - dailies contains wind data but it's more detailed/separated in many columns & incomplete
    'FRSHTT', # Fog, rain, snow, hail, thunderstorm, tornado -> definitely a maybe here
    'DEWP', # Dew point - available in current/historical data
]

First, I'm going to drop the columns that we won't need from the dailies dataframe.  We're mainly looking to keep the *temperature* data: TMIN, TMAX, TAVG.  I will also keep the percipitation (PRCP) in case we want to work with it later.

In [23]:
# Dropping columns that I do not want to use for now

df_dailies_clean = df_dailies.drop(columns=['AWND', 'DAPR', 'FMTM', 'MDPR', 'PGTM', 'SNOW', 'SNWD', 'WDF2', 'WDF5', 'WSF2', 'WSF5'])

df_dailies_clean.head()


,STATION,NAME,DATE,PRCP,TAVG,TMAX,TMIN
0,USW00093820,"LEXINGTON BLUEGRASS AIRPORT, KY US",2000-01-01,0.00,48.0,58.0,38.0
1,USW00093820,"LEXINGTON BLUEGRASS AIRPORT, KY US",2000-01-02,0.25,58.0,64.0,52.0
2,USW00093820,"LEXINGTON BLUEGRASS AIRPORT, KY US",2000-01-03,2.05,61.0,68.0,53.0
3,USW00093820,"LEXINGTON BLUEGRASS AIRPORT, KY US",2000-01-04,0.08,42.0,57.0,27.0
4,USW00093820,"LEXINGTON BLUEGRASS AIRPORT, KY US",2000-01-05,0.00,29.0,32.0,25.0


The dailies dataframe looks much better pared down to a few columns.  Most of these columns also have data in them so there are significantly less NaN, which makes the dataframe just *look* cleaner.

For the other two dataframes, I want to only keep a few columns and also rename the gust columns.  For now I've decided to keep: TEMP, PRCP, GUST_MAX (formally MAX), GUST_MIN (formally MIN), WDSP, FRSHTT, DEWP.

In [55]:
# After spending too long typing all that I don't want.... I figured there's a better way to do it:
# Focus on the columns I DO want.  Less mistakes to be made, and less typing...

def clean_main_data(df):
    """
    This is set up to only keep the columns that I want to use and may use down the line.  It is set up in a way to be used with the two main dataframes.

    I will also be renaming MAX and MIN to GUST_MAX and GUST_MIN, and converting the DATE column to a datetime object.

    ARGS:
        df: The dataframe to clean
    """
    # A list of columns I want to keep, based on the lists above.
    columns_to_keep = [
        'STATION',
        'NAME',
        'DATE',
        'TEMP',
        'PRCP',
        'MAX',
        'MIN',
        'WDSP',
        'FRSHTT',
        'DEWP',        
    ]

    # Dropping columns that I do not want to use for now
    df = df[columns_to_keep]

    # Renaming the columns to be more descriptive
    df = df.rename(columns={'MAX': 'GUST_MAX', 'MIN': 'GUST_MIN'})

    # Converting the DATE column to a datetime object
    df['DATE'] = pd.to_datetime(df['DATE'], format='%Y-%m-%d')

    print("Successfully cleaned the dataframe.")
    return df

df_current_clean = clean_main_data(df_current)
df_current_clean.head()

Successfully cleaned the dataframe.


,STATION,NAME,DATE,TEMP,PRCP,GUST_MAX,GUST_MIN,WDSP,FRSHTT,DEWP
0,72422093820,"LEXINGTON BLUEGRASS AIRPORT, KY US",2000-01-01,46.1,0.00,57.9,30.0,6.8,0,30.0
1,72422093820,"LEXINGTON BLUEGRASS AIRPORT, KY US",2000-01-02,56.7,0.00,64.0,37.9,14.1,0,44.9
2,72422093820,"LEXINGTON BLUEGRASS AIRPORT, KY US",2000-01-03,58.6,1.31,68.0,52.0,11.5,110000,54.1
3,72422093820,"LEXINGTON BLUEGRASS AIRPORT, KY US",2000-01-04,48.2,1.07,68.0,30.0,12.4,111000,42.9
4,72422093820,"LEXINGTON BLUEGRASS AIRPORT, KY US",2000-01-05,28.4,0.08,32.0,26.6,7.0,1000,20.5


In [56]:
df_historical_clean = clean_main_data(df_historical)
df_historical_clean.head()

Successfully cleaned the dataframe.


,STATION,NAME,DATE,TEMP,PRCP,GUST_MAX,GUST_MIN,WDSP,FRSHTT,DEWP
0,72422093820,"LEXINGTON BLUEGRASS AIRPORT, KY US",1973-01-01,37.1,0.00,44.1,30.0,7.4,0,25.3
1,72422093820,"LEXINGTON BLUEGRASS AIRPORT, KY US",1973-01-02,31.2,0.39,37.9,24.1,4.4,0,21.2
2,72422093820,"LEXINGTON BLUEGRASS AIRPORT, KY US",1973-01-03,38.4,0.04,51.1,32.0,11.0,111000,28.1
3,72422093820,"LEXINGTON BLUEGRASS AIRPORT, KY US",1973-01-04,46.5,0.67,55.9,39.9,12.8,10000,38.1
4,72422093820,"LEXINGTON BLUEGRASS AIRPORT, KY US",1973-01-05,32.7,0.00,39.9,30.0,7.2,0,25.9


Now that all of the data is cleaned up, I'm going to check to see if any of the rows/columns have null/NaN data.

In [27]:
df_historical_clean.isnull().sum()

STATION     0
NAME        0
DATE        0
TEMP        0
PRCP        0
GUST_MAX    0
GUST_MIN    0
WDSP        0
FRSHTT      0
DEWP        0
dtype: int64

In [29]:
df_current_clean.isnull().sum()

STATION     0
NAME        0
DATE        0
TEMP        0
PRCP        0
GUST_MAX    0
GUST_MIN    0
WDSP        0
FRSHTT      0
DEWP        0
dtype: int64

In [31]:
df_dailies_clean.isnull().sum()

STATION        0
NAME           0
DATE           0
PRCP           2
TAVG       10834
TMAX           0
TMIN           0
dtype: int64

It looks like dailies has *some* NaN in TAVG.  This was expected based on exploration previously.  Otherwise, it looks pretty complete; just a few PRCP missing which is honestly surprising.  I expected more data to be missing here.  Since there are no missing TMAX and TMIN, I think we could easily create our own AVG by taking the min/max and just averaging the temp that way.  

In [32]:
# Maybe should see what TAVG that are null looks like/where the data comes from
mask = df_dailies_clean['TAVG'].isna()
df_dailies_clean.loc[mask]

,STATION,NAME,DATE,PRCP,TAVG,TMAX,TMIN
289,USW00093820,"LEXINGTON BLUEGRASS AIRPORT, KY US",2000-10-16,0.00,NaN,74.0,54.0
290,USW00093820,"LEXINGTON BLUEGRASS AIRPORT, KY US",2000-10-17,0.10,NaN,70.0,58.0
296,USW00093820,"LEXINGTON BLUEGRASS AIRPORT, KY US",2000-10-23,0.00,NaN,78.0,59.0
297,USW00093820,"LEXINGTON BLUEGRASS AIRPORT, KY US",2000-10-24,0.29,NaN,80.0,62.0
1259,USW00063838,"VERSAILLES 3 NNW, KY US",2003-06-13,0.04,NaN,79.0,68.0
...,...,...,...,...,...,...,...
17363,USW00063838,"VERSAILLES 3 NNW, KY US",2025-07-25,0.00,NaN,91.0,73.0
17365,USW00063838,"VERSAILLES 3 NNW, KY US",2025-07-26,0.00,NaN,92.0,75.0
17366,USW00063838,"VERSAILLES 3 NNW, KY US",2025-07-27,0.20,NaN,92.0,72.0
17369,USW00063838,"VERSAILLES 3 NNW, KY US",2025-07-28,0.50,NaN,92.0,72.0


In [57]:
# I was expecting the missing data to be from any station outside of the BG Airport, but surprisingly they're missing some data too. 
# Just was curious to see where it was from before handling it.
#  
# Handling the TAVG missing data
def fill_tavg(df):
    """
    This function will take a dataframe with TAVG, TMIN, and TMAX columns and fill in the TAVG column with the average of the TMIN and TMAX columns.
    While I know my DF has no null values in TMIN/TMAX, I'm going to try and write this to handle situations where there are null values in these columns.

    ARGS:
        df: The dataframe to clean
    """

    # Creating a mask for locations where TAVG is null (and TMIN and TMAX are not)
    mask = df['TAVG'].isna() & df['TMIN'].notna() & df['TMAX'].notna()

    # Filling in the TAVG column with the average of the TMIN and TMAX columns
    df.loc[mask, 'TAVG'] = (df.loc[mask, 'TMIN'] + df.loc[mask, 'TMAX']) / 2

    print("Successfully filled in the TAVG column.")
    return df

df_dailies_clean = fill_tavg(df_dailies_clean)
df_dailies_clean.isnull().sum()

Successfully filled in the TAVG column.


STATION        0
NAME           0
DATE           0
PRCP           2
TAVG           0
TMAX           0
TMIN           0
DATA_SOURCE    0
YEAR           0
dtype: int64

I'm just going to look at which rows are missing the percipitation data.

In [34]:
# checking prcp that is empty
mask = df_dailies_clean['PRCP'].isna()
df_dailies_clean.loc[mask]

,STATION,NAME,DATE,PRCP,TAVG,TMAX,TMIN
2744,USW00063838,"VERSAILLES 3 NNW, KY US",2005-06-26,NaN,80.5,94.0,67.0
15173,USW00063838,"VERSAILLES 3 NNW, KY US",2022-07-26,NaN,75.0,79.0,71.0


They're both from a station in Versailles.  I am curious if we have any percipitation data for those days from another station

In [35]:
# checking to see if there is more data on the days that are missing PRCP
missing_prcp_dates = df_dailies_clean[df_dailies_clean['PRCP'].isna()]['DATE'].unique()
df_dailies_clean[df_dailies_clean['DATE'].isin(missing_prcp_dates)]

,STATION,NAME,DATE,PRCP,TAVG,TMAX,TMIN
2744,USW00063838,"VERSAILLES 3 NNW, KY US",2005-06-26,NaN,80.5,94.0,67.0
2743,USW00093820,"LEXINGTON BLUEGRASS AIRPORT, KY US",2005-06-26,0.00,81.0,93.0,69.0
15173,USW00063838,"VERSAILLES 3 NNW, KY US",2022-07-26,NaN,75.0,79.0,71.0
15174,USW00093820,"LEXINGTON BLUEGRASS AIRPORT, KY US",2022-07-26,2.03,76.0,83.0,71.0


It looks like we've got percipiation data for each day from another station.  Since our primary focus is Lexington, I think we'll be fine to leave this null data alone.

----

Now that all of the data is cleaned up, I'm going to add some additional labels and combine the two "main" dataframes.  The labels I'll be adding are: 

`DATA_SOURCE` which I will use to separate the data easily down the line.

`YEAR` this will make it easier to fetch/sort by rather than checking the DATE column for the year.

`SEASONS` This will be added to what will become the main dataframe.  I am just going to split the year into seasons based on the months.


In [58]:
# Adding labels and years to the dataframes
# to hopefully make it easier to work with!

def add_labels_and_years(df_historical, df_current, df_dailies):
    """
    This function will add labels and years to the dataframes to make it easier to index and work with in general.
    Since we will also be combining historical and current together, we're creating a column that'll designate which dataframe it came from.

    ARGS:
        df_historical: The historical dataframe
        df_current: The current dataframe
        df_dailies: The dailies dataframe
    """
    # Adding labels to the dataframes
    df_historical['DATA_SOURCE'] = 'historical'
    df_current['DATA_SOURCE'] = 'current'
    df_dailies['DATA_SOURCE'] = 'dailies'

    # Adding years to the dataframes, taking the year from the DATE column
    df_historical['YEAR'] = df_historical['DATE'].dt.year
    df_current['YEAR'] = df_current['DATE'].dt.year
    df_dailies['YEAR'] = df_dailies['DATE'].dt.year

    # Want to rearange this, so these columns are up front
    front_cols = ['DATA_SOURCE', 'STATION', 'NAME', 'DATE', 'YEAR']

    df_historical = df_historical[front_cols + [col for col in df_historical.columns if col not in front_cols]]
    df_current = df_current[front_cols + [col for col in df_current.columns if col not in front_cols]]
    df_dailies = df_dailies[front_cols + [col for col in df_dailies.columns if col not in front_cols]]

    print("Successfully added labels and years to the dataframes.")
    return df_historical, df_current, df_dailies

df_historical_labeled, df_current_labeled, df_dailies_labeled = add_labels_and_years(df_historical_clean, df_current_clean, df_dailies_clean)

df_historical_labeled.head()

Successfully added labels and years to the dataframes.


,DATA_SOURCE,STATION,NAME,DATE,YEAR,TEMP,PRCP,GUST_MAX,GUST_MIN,WDSP,FRSHTT,DEWP
0,historical,72422093820,"LEXINGTON BLUEGRASS AIRPORT, KY US",1973-01-01,1973,37.1,0.00,44.1,30.0,7.4,0,25.3
1,historical,72422093820,"LEXINGTON BLUEGRASS AIRPORT, KY US",1973-01-02,1973,31.2,0.39,37.9,24.1,4.4,0,21.2
2,historical,72422093820,"LEXINGTON BLUEGRASS AIRPORT, KY US",1973-01-03,1973,38.4,0.04,51.1,32.0,11.0,111000,28.1
3,historical,72422093820,"LEXINGTON BLUEGRASS AIRPORT, KY US",1973-01-04,1973,46.5,0.67,55.9,39.9,12.8,10000,38.1
4,historical,72422093820,"LEXINGTON BLUEGRASS AIRPORT, KY US",1973-01-05,1973,32.7,0.00,39.9,30.0,7.2,0,25.9


In [60]:
# One more feature engineering step: adding season to the main dataframes

def add_seasons(df):
    """    
    Seasons based off of the month - Winter, Spring, Summer, Fall
    """
    copy = df.copy()
    # Seasons based off of the DATE column
    copy.loc[:, 'SEASON'] = copy["DATE"].dt.month.map(
        {
            12: "Winter", 1: "Winter", 2: "Winter",
            3: "Spring", 4: "Spring", 5: "Spring",
            6: "Summer", 7: "Summer", 8: "Summer",
            9: "Fall", 10: "Fall", 11: "Fall"
        }
    )

    print("Successfully added seasons to the dataframe.")
    return copy

df_historical_labeled, df_current_labeled = add_seasons(df_historical_labeled), add_seasons(df_current_labeled)

Successfully added seasons to the dataframe.
Successfully added seasons to the dataframe.


In [ ]:
# Now combining the two main dataframes together

def combine_dataframes(df_historical, df_current):
    """
    This function will combine the historical and current dataframes together.
    Will return the dataframe sorted by date.

    ARGS:
        df_historical: The historical dataframe
        df_current: The current dataframe
    """
    # Combining the two dataframes together
    df_combined = pd.concat([df_historical, df_current])

    # Sorting the dataframe by date
    df_combined = df_combined.sort_values(by='DATE')

    print("Successfully combined the two main dataframes.")
    return df_combined

df_combined = combine_dataframes(df_historical_labeled, df_current_labeled)

Successfully combined the two main dataframes.


,DATA_SOURCE,STATION,NAME,DATE,YEAR,TEMP,PRCP,GUST_MAX,GUST_MIN,WDSP,FRSHTT,DEWP,SEASON
9861,historical,99999993820,"LEXINGTON BLUEGRASS AIRPORT, KY US",1948-01-01,1948,53.5,99.99,64.0,37.9,19.9,10000,47.3,Winter
9862,historical,99999993820,"LEXINGTON BLUEGRASS AIRPORT, KY US",1948-01-02,1948,29.1,99.99,35.1,26.1,19.2,111000,27.8,Winter
9863,historical,99999993820,"LEXINGTON BLUEGRASS AIRPORT, KY US",1948-01-03,1948,29.4,99.99,30.9,28.0,6.2,111000,28.6,Winter
9864,historical,99999993820,"LEXINGTON BLUEGRASS AIRPORT, KY US",1948-01-04,1948,32.4,99.99,35.1,30.0,12.8,111000,29.6,Winter
9865,historical,99999993820,"LEXINGTON BLUEGRASS AIRPORT, KY US",1948-01-05,1948,30.9,99.99,34.0,28.0,7.9,1000,27.8,Winter


The next step is to create the database.  I would like the primary key for the tables to be the DATE column, but I know that this will not be possible due to multiple rows of data available per DATE.  I think the main dataframe only has one row per DATE, but I want to check before creating the tables.

In [42]:
if df_combined['DATE'].is_unique:
    print("Yes")
else:
    print("No")

Yes


Looks good! So now I'm going to create the db and tables using these two dataframes: `df_combined` & `df_dailies_labeled`.

Now we can move on to creating the database & tables with these two dataframes.  The goal is to:

- Use DATA as the primary key for the main data and use the default ROWID as the primary key for the dailies table
- Create a few indices that will be beneficial to searching and accessing the database down the line.

In [44]:
# Creating the db and tables

def create_weather_db(df_combined, df_dailies):
    """
    A simple function that is going to create a weather database using the two provided DBs


    ARGS:
        df_combined: The combined dataframe from the historical and current dataframes
        df_dailies: The dailies dataframe
    """
    # Creating & connecting to the database
    DB_PATH = os.path.join('data', 'weather.db')
    conn = sqlite3.connect(DB_PATH)

    # Creating the tables
    df_combined.to_sql('weather_main', conn, if_exists='replace', index=False)
    df_dailies.to_sql('weather_dailies', conn, if_exists='replace', index=False)

    # Creating some ~quick~ indices
    conn.execute("CREATE INDEX IF NOT EXISTS idx_main_date ON weather_main(DATE)")
    conn.execute("CREATE INDEX IF NOT EXISTS idx_dailies_date ON weather_dailies(DATE)")
    conn.execute("CREATE INDEX IF NOT EXISTS idx_source ON weather_main(DATA_SOURCE)")

    conn.close()
    print("Database created successfully")

create_weather_db(df_combined, df_dailies_labeled)

Database created successfully


With the database created successfully, there's one thing left I want to do: save the two dataframes into their own csv files.  This way I'll be able to access the cleaned & sorted data elsewhere if I wanted.

In [45]:
# Saving the cleaned dataframes to new csv files
COMB_FILE_PATH = os.path.join('data', 'combined_weather.csv')
DAILIES_FILE_PATH = os.path.join('data', 'dailies_weather.csv')

df_combined.to_csv(COMB_FILE_PATH, index=False, encoding='utf-8')
df_dailies_clean.to_csv(DAILIES_FILE_PATH, index=False, encoding='utf-8')

# Verifying the files were saved correctly
print(f"Files saved to {COMB_FILE_PATH} and {DAILIES_FILE_PATH}")
print(f"Combined file size: {os.path.getsize(COMB_FILE_PATH) / (1024 * 1024):.2f} MB")
print(f"Dailies file size: {os.path.getsize(DAILIES_FILE_PATH) / (1024 * 1024):.2f} MB")

Files saved to data\combined_weather.csv and data\dailies_weather.csv
Combined file size: 3.13 MB
Dailies file size: 1.46 MB
